# Part 1: Interactive Mapping - Visualizing Sports Radio Networks
## CSC 2053 - Lab 13

Time to put your data on the map! In this lab, you'll create interactive web maps showing sports radio networks across North America.

**What You'll Learn:**
- Creating interactive maps with Folium
- Adding markers with custom popups and tooltips
- Organizing data with layer controls
- Advanced features: clustering, circles, heatmaps
- Building comprehensive geographic dashboards

**What You'll Create:**
- Interactive maps of radio station locations
- Team-specific affiliate network maps
- Power visualization with circle markers
- Format-based layer controls
- A complete sports radio network dashboard



---
## Setup: Load Your Data

Let's load the sports data and prepare for mapping.

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium import plugins

# Choose your sport: "MLB", "NHL", "NFL", or "NBA"
sport = "MLB"  # Change this!

url = f'https://raw.githubusercontent.com/CSC-2053-100-Fall25/python-datascience-template/main/{sport}.csv'
df = pd.read_csv(url)

# Clean data - remove rows with missing coordinates
df_clean = df.dropna(subset=['lat', 'lon'])

print(f"✓ Loaded {sport} data: {len(df_clean)} stations with coordinates")
print(f"✓ Folium ready for mapping!")
print(f"\nCoordinate ranges:")
print(f"  Latitude: {df_clean['lat'].min():.2f} to {df_clean['lat'].max():.2f}")
print(f"  Longitude: {df_clean['lon'].min():.2f} to {df_clean['lon'].max():.2f}")

---
## Part 1: Folium Basics

**What is Folium?** Folium is a Python library that creates interactive web maps using Leaflet.js - the same technology behind many professional mapping websites.

**Why use Folium?**
- **Interactive:** Users can zoom, pan, and click - not just static images
- **Web-ready:** Maps export as HTML files that work in any browser
- **No JavaScript needed:** Create sophisticated maps with just Python
- **Professional:** Same quality as commercial mapping applications

**When to use Folium:**
- Visualizing geographic data (anything with lat/lon)
- Creating interactive dashboards
- Sharing maps with non-technical users
- Exploratory geographic analysis
- Presentations and reports

**Key concept:** Folium bridges Python data analysis and interactive web visualization - analyze in Python, interact in the browser!

**Cool feature:** Your maps work offline once exported to HTML!

### Your First Map

In [ ]:
# Create a basic map centered on USA
m = folium.Map(
    location=[39.8283, -98.5795],  # Center of USA
    zoom_start=4,
    tiles='OpenStreetMap'
)

m

### Adding Markers

In [ ]:
# Map with first 10 stations
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Add markers for first 10 stations
for idx, row in df_clean.head(10).iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['callsign']} - {row['city']}, {row['state']}",
        tooltip=row['callsign']
    ).add_to(m)

m

### Different Map Styles

In [ ]:
# Try different tile sets
m = folium.Map(
    location=[39.8283, -98.5795],
    zoom_start=4,
    tiles='CartoDB positron'  # Clean, minimal style
)

# Add a few markers
for idx, row in df_clean.head(20).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        popup=row['callsign'],
        color='blue',
        fill=True,
        fillColor='blue'
    ).add_to(m)

m

### Exercise 1.1: Create Your First Map

Create a map showing stations from your home state/province.

In [ ]:
# YOUR CODE HERE

# 1. Filter for stations in your home state (e.g., "PA", "CA", "TX")
home_state = "PA"  # Change this
home_stations = 

# 2. Create a map centered on those stations
# Hint: Use .mean() for lat/lon to find center

# 3. Add markers for all stations in your state
# Popup should show: callsign, frequency, and city

# 4. Display the map


---
## Part 2: Marker Customization

**Why customize markers?** Default pins are boring and uninformative. Custom markers turn your map from a dot collection into a rich data visualization.

**What you can customize:**
- **Colors:** Show categories or values (format types, power levels)
- **Sizes:** Represent magnitude (power, coverage area)
- **Icons:** Use symbols that match your data (towers, buildings, logos)
- **Popups:** Show detailed information on click
- **Tooltips:** Display quick info on hover

**When to customize:**
- Multiple categories need visual distinction
- Values have different magnitudes
- Users need detailed information about each point
- You want to tell a visual story

**Design principles:**
- **Color:** Use consistently (same category = same color)
- **Size:** Bigger = more important/larger value
- **Info:** Most important in tooltip, details in popup

**Pro tip:** HTML in popups lets you create rich, formatted information cards!

### Rich Popups with HTML

In [ ]:
# Create detailed popups with HTML
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Add markers with rich popups
for idx, row in df_clean.head(50).iterrows():
    popup_html = f"""
    <div style="font-family: Arial; width: 200px;">
        <h4 style="margin-bottom: 5px; color: #2c3e50;">{row['callsign']}</h4>
        <hr style="margin: 5px 0;">
        <p style="margin: 3px 0;"><b>Frequency:</b> {row['frequency']} MHz</p>
        <p style="margin: 3px 0;"><b>City:</b> {row['city']}, {row['state']}</p>
        <p style="margin: 3px 0;"><b>Format:</b> {row['new_format']}</p>
        <p style="margin: 3px 0;"><b>Power:</b> {row['erp']:.1f} kW</p>
        <p style="margin: 3px 0;"><b>Owner:</b> {row['owner']}</p>
    </div>
    """
    
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['callsign']} - {row['frequency']} MHz",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

m

### Color-Coded Markers by Format

In [ ]:
# Define color mapping for formats
format_colors = {
    'Sports': 'red',
    'News/Talk': 'blue',
    'Country': 'green',
    'Classic Rock': 'purple',
    'Top 40': 'orange',
}

def get_format_color(format_name):
    """Return color for format, default to gray"""
    return format_colors.get(format_name, 'gray')

m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Add color-coded markers
for idx, row in df_clean.head(100).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        popup=f"{row['callsign']} - {row['new_format']}",
        tooltip=row['callsign'],
        color=get_format_color(row['new_format']),
        fill=True,
        fillColor=get_format_color(row['new_format']),
        fillOpacity=0.7
    ).add_to(m)

m

### Power Visualization with Circle Size

In [ ]:
# Circle size represents power
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Normalize power for circle radius (1-20 range)
max_power = df_clean['erp'].max()

for idx, row in df_clean.head(200).iterrows():
    # Scale radius based on power
    radius = 3 + (row['erp'] / max_power) * 15
    
    # Color by power (low = green, high = red)
    power_ratio = row['erp'] / max_power
    if power_ratio < 0.2:
        color = 'green'
    elif power_ratio < 0.5:
        color = 'orange'
    else:
        color = 'red'
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=f"{row['callsign']}<br>Power: {row['erp']:.1f} kW",
        tooltip=f"{row['callsign']} ({row['erp']:.1f} kW)",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6
    ).add_to(m)

m

### Exercise 2.1: Custom Marker Styling

Create a map with sophisticated marker customization.

In [ ]:
# YOUR CODE HERE

# Create a map showing the top 100 most powerful stations with:
# 1. Circle markers sized by power
# 2. Colors based on format (create your own color scheme)
# 3. Rich HTML popups showing all key information
# 4. Tooltips with callsign and power

# Filter for top 100 by power
top_power = 

# Create your map


---
## Part 3: Layer Controls

**What are layers?** Layers let users toggle different data groups on and off - like layers in Photoshop or Google Maps' traffic/transit options.

**Why use layers?**
- **Reduce clutter:** Show only what's relevant at the moment
- **Enable comparison:** Turn layers on/off to compare patterns
- **User control:** Let viewers explore data their way
- **Organization:** Group related data logically

**When to use layers:**
- Multiple categories or data types (formats, owners, teams)
- Optional information (labels, boundaries, overlays)
- Comparison scenarios ("Show me Format A vs Format B")
- Complex datasets with many dimensions

**Layer strategies:**
- **By category:** One layer per format, owner, or team
- **By criteria:** "High power", "Low power", "Medium power"
- **By geography:** State layers, market layers
- **Combination:** Mix base maps and data layers

**Key feature:** Layer controls appear as a toggle box on your map - users click to show/hide!

### Feature Groups

In [ ]:
# Create layers for different formats
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Get top 5 formats
top_formats = df_clean['new_format'].value_counts().head(5).index

# Create a feature group for each format
for format_name in top_formats:
    feature_group = folium.FeatureGroup(name=format_name)
    
    # Add stations with this format
    format_stations = df_clean[df_clean['new_format'] == format_name]
    
    for idx, row in format_stations.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            popup=f"{row['callsign']}<br>{row['city']}, {row['state']}",
            tooltip=row['callsign'],
            color=get_format_color(format_name),
            fill=True,
            fillColor=get_format_color(format_name),
            fillOpacity=0.7
        ).add_to(feature_group)
    
    feature_group.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

### Multiple Base Maps

In [ ]:
# Map with multiple tile options
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4,
    tiles='OpenStreetMap'
)

# Add alternative tile layers
folium.TileLayer('CartoDB positron', name='Light Map').add_to(m)
folium.TileLayer('CartoDB dark_matter', name='Dark Map').add_to(m)

# Add some markers
for idx, row in df_clean.head(100).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=4,
        popup=row['callsign'],
        color='red',
        fill=True
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

---
## Part 4: Advanced Features

**What are advanced features?** Specialized visualization techniques that go beyond simple markers: clustering, heatmaps, circles, and more.

**Why go advanced?**
- **Handle scale:** Thousands of markers need clustering
- **Show patterns:** Heatmaps reveal density and hotspots
- **Represent coverage:** Circles show ranges and areas of influence
- **Professional polish:** Advanced features look more sophisticated

**Feature overview:**

**Marker Clusters:**
- **Use for:** Large datasets (100+ points)
- **Benefit:** Automatic grouping at different zoom levels
- **Example:** "All 821 MLB affiliate stations"

**Heatmaps:**
- **Use for:** Showing density and concentration
- **Benefit:** See patterns that individual points hide
- **Example:** "Where are stations most concentrated?"

**Circles:**
- **Use for:** Coverage areas or influence zones
- **Benefit:** Show reach and overlap
- **Example:** "Broadcast coverage radius"

**Pro tip:** Combine techniques - clusters for overview, individual markers with circles for detail!

### Marker Clusters

In [ ]:
# Cluster markers to reduce clutter
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Create marker cluster
marker_cluster = plugins.MarkerCluster().add_to(m)

# Add ALL stations to cluster
for idx, row in df_clean.iterrows():
    popup_html = f"""
    <b>{row['callsign']}</b><br>
    {row['frequency']} MHz<br>
    {row['city']}, {row['state']}<br>
    {row['new_format']}<br>
    Power: {row['erp']:.1f} kW
    """
    
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup_html,
        tooltip=row['callsign']
    ).add_to(marker_cluster)

m

### Circle Markers for Coverage Areas

In [ ]:
# Show approximate coverage area with circles
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=5
)

# Add circles for high-power stations
high_power = df_clean[df_clean['erp'] > 50].head(20)

for idx, row in high_power.iterrows():
    # Radius in meters (rough approximation based on power)
    # Higher power = larger coverage
    radius_meters = np.sqrt(row['erp']) * 5000
    
    # Coverage circle
    folium.Circle(
        location=[row['lat'], row['lon']],
        radius=radius_meters,
        popup=f"{row['callsign']}<br>Coverage: ~{radius_meters/1000:.0f} km",
        tooltip=row['callsign'],
        color='blue',
        fill=True,
        fillOpacity=0.2
    ).add_to(m)
    
    # Station marker
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['callsign']}<br>Power: {row['erp']:.0f} kW",
        icon=folium.Icon(color='red', icon='tower-broadcast', prefix='fa')
    ).add_to(m)

m

### Heatmap

In [ ]:
# Show station density with heatmap
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Prepare data for heatmap
heat_data = [[row['lat'], row['lon']] for idx, row in df_clean.iterrows()]

# Add heatmap
plugins.HeatMap(heat_data, radius=15, blur=25).add_to(m)

m

### Exercise 4.1: Advanced Visualization

Combine multiple advanced features in one map.

In [ ]:
# YOUR CODE HERE

# Create a map that shows:
# 1. Marker clusters for ALL stations
# 2. Coverage circles for the top 10 most powerful stations
# 3. Different marker colors based on format
# 4. Rich HTML popups with all station info

# This combines clustering, circles, and custom markers!


---
## Part 5: Team Network Maps

**What are network maps?** Visualizations showing all affiliates that broadcast a specific team's games - the geographic reach of a sports franchise.

**Why map team networks?**
- **Understand reach:** See how far a team's fan base extends
- **Market analysis:** Identify strong and weak coverage areas
- **Strategy:** Plan expansion or partnership opportunities
- **Storytelling:** Show the power of sports broadcasting

**What network maps reveal:**
- Geographic coverage (which states/regions)
- Market penetration (major cities vs rural areas)
- Power distribution (where are the strongest signals?)
- Ownership patterns (who controls the network?)

**Use cases:**
- **Teams:** "Where can fans listen to our games?"
- **Advertisers:** "What's our potential reach?"
- **Leagues:** "Which franchises have the biggest radio presence?"
- **Researchers:** "How does geography affect fan loyalty?"

**Comparison power:** Overlay multiple teams to see market competition and overlap!

### Find Teams in Your Data

In [ ]:
# Identify which team column to use
team_column = sport.lower()  # 'mlb', 'nhl', 'nfl', or 'nba'

# Get all teams with affiliates
teams = df_clean[team_column].dropna().unique()
print(f"Teams with radio affiliates ({len(teams)}):")
for team in sorted(teams):
    count = len(df_clean[df_clean[team_column] == team])
    print(f"  {team}: {count} stations")

### Single Team Network

In [ ]:
# Map one team's affiliate network
team_name = teams[0]  # Pick first team, or change this
team_affiliates = df_clean[df_clean[team_column] == team_name]

print(f"Mapping {team_name} network: {len(team_affiliates)} affiliates")

m = folium.Map(
    location=[team_affiliates['lat'].mean(), team_affiliates['lon'].mean()],
    zoom_start=5
)

# Add markers for each affiliate
for idx, row in team_affiliates.iterrows():
    popup_html = f"""
    <div style="width: 220px; font-family: Arial;">
        <h3 style="color: #2c3e50; margin-bottom: 5px;">{row['callsign']}</h3>
        <p style="margin: 2px 0; font-size: 14px; font-weight: bold; color: #e74c3c;">
            {team_name} Affiliate
        </p>
        <hr style="margin: 5px 0;">
        <p style="margin: 3px 0;"><b>Frequency:</b> {row['frequency']} MHz</p>
        <p style="margin: 3px 0;"><b>Location:</b> {row['city']}, {row['state']}</p>
        <p style="margin: 3px 0;"><b>Format:</b> {row['new_format']}</p>
        <p style="margin: 3px 0;"><b>Power:</b> {row['erp']:.1f} kW</p>
        <p style="margin: 3px 0;"><b>Owner:</b> {row['owner']}</p>
    </div>
    """
    
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['callsign']} - {row['city']}",
        icon=folium.Icon(color='red', icon='broadcast-tower', prefix='fa')
    ).add_to(m)

# Add title
title_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 300px; height: 60px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:16px; text-align: center; padding: 10px;">
        <h3 style="margin-bottom: 5px;">{team_name}</h3>
        <p style="margin: 0;">Radio Affiliate Network</p>
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

m

### Multi-Team Comparison

In [ ]:
# Compare networks of multiple teams
teams_to_compare = teams[:3]  # First 3 teams
team_colors = ['red', 'blue', 'green', 'purple', 'orange']

m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4
)

# Create layer for each team
for i, team in enumerate(teams_to_compare):
    team_affiliates = df_clean[df_clean[team_column] == team]
    color = team_colors[i]
    
    feature_group = folium.FeatureGroup(name=f"{team} ({len(team_affiliates)} stations)")
    
    for idx, row in team_affiliates.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=6,
            popup=f"{row['callsign']}<br>{team}<br>{row['city']}, {row['state']}",
            tooltip=f"{row['callsign']} ({team})",
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(feature_group)
    
    feature_group.add_to(m)

folium.LayerControl().add_to(m)

m

### Exercise 5.1: Team Network Analysis

Create a comprehensive map of your favorite team's affiliate network.

In [ ]:
# YOUR CODE HERE

# Pick your favorite team and create a map showing:
# 1. All affiliate stations
# 2. Circle markers sized by power
# 3. Coverage circles for the most powerful affiliates
# 4. Rich popups with all station details
# 5. A title banner identifying the team

# Bonus: Color-code by format to see what types of stations carry the games

favorite_team = ""  # Fill this in!


---
## Part 6: The Big Picture - All Four Sports

**Let's bring it all together!** Instead of looking at one sport at a time, let's see the complete landscape of sports radio across North America.

**What we'll discover:**
- Which regions have the most sports coverage?
- How do different sports compare in their radio presence?
- Are there patterns in where teams build their networks?
- Which sports have the widest geographic reach?

**The power of multi-dataset analysis:** Sometimes the most interesting insights come from comparing across categories!

### Step 1: Load All Four Sports

Let's load all four datasets and combine them into one master dataset.

In [ ]:
# Load all four sport datasets
sports_list = ['MLB', 'NHL', 'NFL', 'NBA']
all_sports_data = []

for sport in sports_list:
    url = f'https://raw.githubusercontent.com/CSC-2053-100-Fall25/python-datascience-template/main/{sport}.csv'
    df_sport = pd.read_csv(url)
    
    # Add a column to identify which sport this data is from
    df_sport['sport'] = sport
    
    # Clean data - remove rows with missing coordinates
    df_sport = df_sport.dropna(subset=['lat', 'lon'])
    
    all_sports_data.append(df_sport)
    print(f"✓ Loaded {sport}: {len(df_sport)} stations")

# Combine all sports into one DataFrame
all_sports = pd.concat(all_sports_data, ignore_index=True)

print(f"\n✓ Total stations across all sports: {len(all_sports)}")
print(f"✓ Ready to map the complete sports radio landscape!")

### Step 2: Create the Master Sports Map

Now let's create an interactive map with a separate layer for each sport. You'll be able to toggle sports on and off to compare!

In [ ]:
# Define colors for each sport
sport_colors = {
    'MLB': 'red',
    'NFL': 'blue', 
    'NBA': 'orange',
    'NHL': 'purple'
}

# Create the base map
m = folium.Map(
    location=[all_sports['lat'].mean(), all_sports['lon'].mean()],
    zoom_start=4,
    tiles='CartoDB positron'
)

# Create a feature group (layer) for each sport
for sport in sports_list:
    sport_data = all_sports[all_sports['sport'] == sport]
    color = sport_colors[sport]
    
    # Create layer for this sport
    feature_group = folium.FeatureGroup(name=f"{sport} ({len(sport_data)} stations)")
    
    # Add markers for each station
    for idx, row in sport_data.iterrows():
        popup_html = f"""
        <div style="font-family: Arial; width: 220px;">
            <h4 style="margin-bottom: 5px; color: {color};">{row['callsign']}</h4>
            <p style="margin: 2px 0; font-size: 14px; font-weight: bold;">
                {sport} Affiliate
            </p>
            <hr style="margin: 5px 0;">
            <p style="margin: 3px 0;"><b>Frequency:</b> {row['frequency']} MHz</p>
            <p style="margin: 3px 0;"><b>Location:</b> {row['city']}, {row['state']}</p>
            <p style="margin: 3px 0;"><b>Format:</b> {row['new_format']}</p>
            <p style="margin: 3px 0;"><b>Power:</b> {row['erp']:.1f} kW</p>
        </div>
        """
        
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"{row['callsign']} ({sport})",
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.6
        ).add_to(feature_group)
    
    feature_group.add_to(m)

# Add layer control so users can toggle sports on/off
folium.LayerControl(collapsed=False).add_to(m)

# Add a title
title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 350px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; text-align: center; padding: 10px;">
        <h3 style="margin-bottom: 5px;">Sports Radio Landscape</h3>
        <p style="margin: 0; font-size: 12px;">MLB • NFL • NBA • NHL</p>
        <p style="margin: 5px 0; font-size: 11px; color: #666;">
            Use layer controls (top right) to toggle sports
        </p>
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

print("✓ Map created! Explore by toggling layers on and off")
m

### Step 3: Quick Statistics

Let's see some basic statistics about each sport's radio presence.

In [ ]:
# Summary statistics by sport
print("Sports Radio Statistics:\n")

for sport in sports_list:
    sport_data = all_sports[all_sports['sport'] == sport]
    
    print(f"{sport}:")
    print(f"  Total Stations: {len(sport_data)}")
    print(f"  Average Power: {sport_data['erp'].mean():.1f} kW")
    print(f"  Max Power: {sport_data['erp'].max():.1f} kW")
    print(f"  States Covered: {sport_data['state'].nunique()}")
    print()

---
## Reflection: What Does the Map Tell Us?

Now that you've explored the complete sports radio landscape, take some time to analyze what you see.

**Instructions:**
1. Toggle different sports layers on and off
2. Zoom into different regions of the country
3. Click on markers to see station details
4. Look for patterns, concentrations, and interesting observations

**Write three key takeaways from the map below.** Consider questions like:
- Which sport has the most extensive radio network?
- Are there regional patterns (e.g., more hockey in the north, more baseball in certain areas)?
- Which regions have the most sports radio coverage overall?
- Do you see any surprising gaps or concentrations?
- How does your home region compare to others?

### Your Three Takeaways:

**Takeaway 1:**

[Write your first observation here]

---

**Takeaway 2:**

[Write your second observation here]

---

**Takeaway 3:**

[Write your third observation here]

---

---
## Wrap-Up
### Real Skills You've Gained:
- Create interactive web-based visualizations
- Map geographic data effectively
- Build layered, complex dashboards
- Tell stories with spatial data
- Analyze geographic patterns and distributions

### Folium Best Practices:
1. **Use clustering** for large datasets (>100 markers)
2. **Layer organization** - Group related data together
3. **Rich popups** - Include all relevant information
4. **Tooltips** - Show key info on hover
5. **Visual hierarchy** - Use size and color to show importance
6. **Test zoom levels** - Ensure map works at different scales

### Taking It Further:
- Export your maps as HTML files to share
- Embed maps in web applications
- Explore Plotly for even more interactive features
- Combine with other libraries (GeoPandas) for advanced GIS

### What You've Accomplished:
Over these 4 labs, you've gone from Python basics to creating sophisticated, interactive data analysis dashboards. You can now:
- Clean and process real-world datasets
- Perform complex data analysis with Pandas
- Create professional visualizations
- Build interactive geographic dashboards
- Tell compelling stories with data

**You're thinking like a data scientist!** 🗺️📊🎯

---

### Save Your Work

Remember to save your favorite maps:

In [ ]:
# Save any map to an HTML file
# Example:
# m.save('my_sports_radio_map.html')

# Then open the file in any web browser to view and share!